In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls '/content/drive/My Drive/Colab Notebooks/Twitter.properties'

'/content/drive/My Drive/Colab Notebooks/Twitter.properties'


In [3]:
!pip install ConfigParser

In [6]:
import configparser
config=configparser.RawConfigParser()
config.read('/content/drive/My Drive/Colab Notebooks/Twitter.properties')

print(config.sections())

['twitter']


In [7]:
import os
import tweepy as tp

In [9]:
accesstoken=config.get('twitter','accesstoken')
accesstokensecret=config.get('twitter','accesstokensecret')
apikey=config.get('twitter','apikey')
apisecretkey=config.get('twitter','apisecretkey')

In [10]:
auth=tp.OAuthHandler(apikey,apisecretkey)
auth.set_access_token(accesstoken,accesstokensecret)
api=tp.API(auth,wait_on_rate_limit=True)

In [12]:
search_words=["#ipl2021","#RRvPBKS"]
date_since="2021-04-04"

In [14]:
tweets=tp.Cursor(api.search, q=search_words, lang="en", since=date_since).items(1000)

In [15]:
tweets

In [16]:
tweet_details=[[tweet.geo, tweet.text, tweet.user.screen_name, tweet.user.location] for tweet in tweets]

In [17]:
import pandas as pd

In [19]:
tweet_df= pd.DataFrame(data=tweet_details, columns=['Geo','Text','User_name','Location'])

In [20]:
pd.set_option('max_colwidth',800)
tweet_df.head(25)

,Geo,Text,User_name,Location
0,None,@ViCustomerCare @DisneyplusHSVIP Ans19) 3 singles\n\n#FanOfTheMatch #RRvPBKS #IPL2021\n#Vi @ViCustomerCare,vimlesh_321,
1,None,@ViCustomerCare 54 balls\n\n#FanOfTheMatch #RRvPBKS #IPL2021 @ViCustomerCare \n\n💖 💖\n 💖 💖\n 💖 💖\n 💖… https://t.co/a3sTSdZUF4,MBatra01,"Lucknow, India"
2,None,RT @WasimJaffer14: Yes. Here's the match summary 😅\n#RRvPBKS #IPL2021 https://t.co/QbQNUedxiN https://t.co/ZOIwy3PvS0,_hardie__,♡
3,None,"RT @ViCustomerCare: Answer correctly in the comments section &amp; you can win an iPhone. Also, keep an eye out for the next question.\nTip: Str…",vimlesh_321,
4,None,@ViCustomerCare @DisneyplusHSVIP 3 Singles\n\n#FanOfTheMatch #RRvPBKS #IPL2021,PrernaJ54110576,"Ludhiana, Punjab, India"
5,None,"RT @PunjabKingsIPL: ""I was just reacting to the ball and showing aggressive intent in my batting!"" - @HoodaOnFire \n\n #SaddaPunjab #PunjabKi…",anupampatel15,"Uttar Pradesh, India"
6,None,RT @WasimJaffer14: Yes. Here's the match summary 😅\n#RRvPBKS #IPL2021 https://t.co/ZOIwy3PvS0,amirjamal0,India
7,None,When Morris was denied Single by Samson. #RRvPBKS #Ipl2021 \n😂😂 https://t.co/iNLe3lxKvW,nitesh632,"Jaipur, India"
8,None,"RT @StarSportsIndia: Gotta admit, this was 𝑾𝑨𝒀 more excitement than we are used to on a Monday. 😅\n\nRelive the best of #RRvPBKS with these 📸…",kavi_starks,
9,None,RT @PunjabKingsIPL: 💔💔💔\n\nKL Rahul walks back for a stunning 91 (50) 🙌👏\n\n#SaddaPunjab #PunjabKings #IPL2021 #RRvPBKS,ShyamKumarr1997,"Nagpur, India"


In [22]:
tweet_df.User_name.value_counts()

sonika_khambra     33
aakash_beast786    13
MBatra01           11
THEMURTUZA         11
raju_sebin         10
                   ..
Darling7723         1
ImNareshKasturi     1
BaswarajJangam1     1
Ankithatp           1
Sathya46817012      1
Name: User_name, Length: 626, dtype: int64

In [23]:
tweet_df.Location.value_counts()

                                304
India                            78
Mumbai, India                    38
Bangalore                        33
Kerala, India                    21
                               ... 
Sonarpur, Kolkata                 1
🧡                                 1
Thalapathy Fort--- TUTICORIN      1
Bahrain                           1
London                            1
Name: Location, Length: 253, dtype: int64

In [29]:
import re
def clean_tweets(text):
  text=re.sub("RT @[\w]*:","",text)
  text=re.sub("@[\w]*","",text)
  text=re.sub("https?://[A-Za-z0-9./]*","",text)
  text=re.sub("\n","",text)
  return text

In [30]:
tweet_df['Text']=tweet_df['Text'].apply(lambda x: clean_tweets(x))

In [31]:
tweet_df.head(20)

,Geo,Text,User_name,Location
0,None,Ans19) 3 singles#FanOfTheMatch #RRvPBKS #IPL2021#Vi,vimlesh_321,
1,None,54 balls#FanOfTheMatch #RRvPBKS #IPL2021 💖 💖 💖 💖 💖 💖 💖…,MBatra01,"Lucknow, India"
2,None,Yes. Here's the match summary 😅#RRvPBKS #IPL2021,_hardie__,♡
3,None,"Answer correctly in the comments section &amp; you can win an iPhone. Also, keep an eye out for the next question.Tip: Str…",vimlesh_321,
4,None,3 Singles#FanOfTheMatch #RRvPBKS #IPL2021,PrernaJ54110576,"Ludhiana, Punjab, India"
5,None,"""I was just reacting to the ball and showing aggressive intent in my batting!"" - #SaddaPunjab #PunjabKi…",anupampatel15,"Uttar Pradesh, India"
6,None,Yes. Here's the match summary 😅#RRvPBKS #IPL2021,amirjamal0,India
7,None,When Morris was denied Single by Samson. #RRvPBKS #Ipl2021 😂😂,nitesh632,"Jaipur, India"
8,None,"Gotta admit, this was 𝑾𝑨𝒀 more excitement than we are used to on a Monday. 😅Relive the best of #RRvPBKS with these 📸…",kavi_starks,
9,None,💔💔💔KL Rahul walks back for a stunning 91 (50) 🙌👏#SaddaPunjab #PunjabKings #IPL2021 #RRvPBKS,ShyamKumarr1997,"Nagpur, India"


In [32]:
tweet_df.to_csv('tweets_ipl.csv')

In [33]:
!ls

drive  sample_data  tweets_ipl.csv


In [34]:
import spacy
nlp=spacy.load('en_core_web_sm')

In [36]:
tweet_df['Text'].apply(lambda x:[print("\tText : {}, Entity : {}".format(ent.text, ent.label_))
      if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])

	Text : 3, Entity : CARDINAL
	Text : 54, Entity : CARDINAL
	Text : Answer, Entity : ORG
	Text : iPhone, Entity : ORG
	Text : 3, Entity : CARDINAL
	Text : Morris, Entity : PERSON
	Text : Samson, Entity : PERSON
	Text : 𝑾𝑨𝒀, Entity : ORG
	Text : Monday, Entity : DATE
	Text : RRvPBKS, Entity : MONEY
	Text : 91, Entity : CARDINAL
	Text : 50, Entity : CARDINAL
	Text : 👏#SaddaPunjab #PunjabKings #IPL2021, Entity : ORG
	Text : 200, Entity : CARDINAL
	Text : IPL, Entity : GPE
	Text : 299, Entity : CARDINAL
	Text : one, Entity : CARDINAL
	Text : tonight, Entity : TIME
	Text : one, Entity : CARDINAL
	Text : Sanju Samson, Entity : PERSON
	Text : Scores, Entity : CARDINAL
	Text : 102, Entity : CARDINAL
	Text : 54, Entity : CARDINAL
	Text : Answer, Entity : ORG
	Text : iPhone, Entity : ORG
	Text : Punjab Kings, Entity : PERSON
	Text : four, Entity : CARDINAL
	Text : KL Rahul, Entity : PERSON
	Text : Hooda, Entity : PERSON
	Text : Samson, Entity : PERSON
	Text : 200, Entity : CARDINAL
	Text : IPL, E

0              [None]
1              [None]
2                  []
3        [None, None]
4              [None]
            ...      
995            [None]
996                []
997      [None, None]
998    [None, None, ]
999            [None]
Name: Text, Length: 1000, dtype: object

In [37]:
tweet_df['entities']=tweet_df['Text'].apply(lambda x:[(ent.text, ent.label_)
      if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])

In [38]:
tweet_df.head(20)

,Geo,Text,User_name,Location,entities
0,None,Ans19) 3 singles#FanOfTheMatch #RRvPBKS #IPL2021#Vi,vimlesh_321,,"[(3, CARDINAL)]"
1,None,54 balls#FanOfTheMatch #RRvPBKS #IPL2021 💖 💖 💖 💖 💖 💖 💖…,MBatra01,"Lucknow, India","[(54, CARDINAL)]"
2,None,Yes. Here's the match summary 😅#RRvPBKS #IPL2021,_hardie__,♡,[]
3,None,"Answer correctly in the comments section &amp; you can win an iPhone. Also, keep an eye out for the next question.Tip: Str…",vimlesh_321,,"[(Answer, ORG), (iPhone, ORG)]"
4,None,3 Singles#FanOfTheMatch #RRvPBKS #IPL2021,PrernaJ54110576,"Ludhiana, Punjab, India","[(3, CARDINAL)]"
5,None,"""I was just reacting to the ball and showing aggressive intent in my batting!"" - #SaddaPunjab #PunjabKi…",anupampatel15,"Uttar Pradesh, India",[]
6,None,Yes. Here's the match summary 😅#RRvPBKS #IPL2021,amirjamal0,India,[]
7,None,When Morris was denied Single by Samson. #RRvPBKS #Ipl2021 😂😂,nitesh632,"Jaipur, India","[(Morris, PERSON), (Samson, PERSON)]"
8,None,"Gotta admit, this was 𝑾𝑨𝒀 more excitement than we are used to on a Monday. 😅Relive the best of #RRvPBKS with these 📸…",kavi_starks,,"[(𝑾𝑨𝒀, ORG), (Monday, DATE), (RRvPBKS, MONEY)]"
9,None,💔💔💔KL Rahul walks back for a stunning 91 (50) 🙌👏#SaddaPunjab #PunjabKings #IPL2021 #RRvPBKS,ShyamKumarr1997,"Nagpur, India","[(91, CARDINAL), (50, CARDINAL), (👏#SaddaPunjab #PunjabKings #IPL2021, ORG)]"


In [39]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [41]:
sid=SentimentIntensityAnalyzer()

In [42]:
tweet_df['sentiment']=tweet_df['Text'].apply(lambda x:sid.polarity_scores(x))

In [43]:
tweet_df.head(20)

,Geo,Text,User_name,Location,entities,sentiment
0,None,Ans19) 3 singles#FanOfTheMatch #RRvPBKS #IPL2021#Vi,vimlesh_321,,"[(3, CARDINAL)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
1,None,54 balls#FanOfTheMatch #RRvPBKS #IPL2021 💖 💖 💖 💖 💖 💖 💖…,MBatra01,"Lucknow, India","[(54, CARDINAL)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
2,None,Yes. Here's the match summary 😅#RRvPBKS #IPL2021,_hardie__,♡,[],"{'neg': 0.0, 'neu': 0.69, 'pos': 0.31, 'compound': 0.4019}"
3,None,"Answer correctly in the comments section &amp; you can win an iPhone. Also, keep an eye out for the next question.Tip: Str…",vimlesh_321,,"[(Answer, ORG), (iPhone, ORG)]","{'neg': 0.0, 'neu': 0.847, 'pos': 0.153, 'compound': 0.5859}"
4,None,3 Singles#FanOfTheMatch #RRvPBKS #IPL2021,PrernaJ54110576,"Ludhiana, Punjab, India","[(3, CARDINAL)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
5,None,"""I was just reacting to the ball and showing aggressive intent in my batting!"" - #SaddaPunjab #PunjabKi…",anupampatel15,"Uttar Pradesh, India",[],"{'neg': 0.112, 'neu': 0.888, 'pos': 0.0, 'compound': -0.2244}"
6,None,Yes. Here's the match summary 😅#RRvPBKS #IPL2021,amirjamal0,India,[],"{'neg': 0.0, 'neu': 0.69, 'pos': 0.31, 'compound': 0.4019}"
7,None,When Morris was denied Single by Samson. #RRvPBKS #Ipl2021 😂😂,nitesh632,"Jaipur, India","[(Morris, PERSON), (Samson, PERSON)]","{'neg': 0.244, 'neu': 0.756, 'pos': 0.0, 'compound': -0.4404}"
8,None,"Gotta admit, this was 𝑾𝑨𝒀 more excitement than we are used to on a Monday. 😅Relive the best of #RRvPBKS with these 📸…",kavi_starks,,"[(𝑾𝑨𝒀, ORG), (Monday, DATE), (RRvPBKS, MONEY)]","{'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'compound': 0.8588}"
9,None,💔💔💔KL Rahul walks back for a stunning 91 (50) 🙌👏#SaddaPunjab #PunjabKings #IPL2021 #RRvPBKS,ShyamKumarr1997,"Nagpur, India","[(91, CARDINAL), (50, CARDINAL), (👏#SaddaPunjab #PunjabKings #IPL2021, ORG)]","{'neg': 0.0, 'neu': 0.809, 'pos': 0.191, 'compound': 0.3818}"
